In [ ]:
%%time

import os
import sys

sys.path.append('..')

from movies_dask_bag.movie_reader import TheatersReader, MoviesReader, ShowingsReader

work_dir = os.environ.get('SLURM_TMPDIR', '.')
data_dir = '{}/json'.format(work_dir)
file_pattern = '{}/*/*'.format(data_dir)

showings_reader = ShowingsReader(file_pattern)

showings_reader.take(1)

In [ ]:
showings_reader.client

In [ ]:
showings_reader.bag

In [ ]:
%%time

showings_reader.count

In [ ]:
%%time

a = showings_reader.bag.map(lambda x: x['country'] + '--' + x['date_stamp']).frequencies(sort=False).compute()

In [ ]:
len(a)

In [ ]:
a[:5]

In [ ]:
date_counts = showings_reader.bag.map(lambda x: x['date_stamp']).frequencies(sort=False).compute()

In [ ]:
columns = sorted([date_count[0] for date_count in date_counts])
columns

In [ ]:
country_counts = showings_reader.bag.map(lambda x: x['country']).frequencies(sort=False).compute()
countries = sorted([country_count[0] for country_count in country_counts])
countries

In [ ]:
len(countries)

In [ ]:
import pandas as pd
import numpy as np

# df = pd.DataFrame(columns=columns, index=['country'], dtype=np.int64)
df = pd.DataFrame(columns=columns, dtype=np.int64)
df

In [ ]:
for row in a:
    (country, date_stamp) = row[0].split('--')
    df.loc[country, date_stamp] = row[1]
df

In [ ]:
import datetime

columns = [datetime.datetime.strptime(c, '%Y%m%d') for c in columns]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
for country in countries:
    fig.add_scatter(x=columns,
                    y=df.loc[country],
                    mode = 'lines',
                    name=country)

fig.show()

In [ ]:
df_ratio = df.apply(lambda x:x / df['20191213']) * 100

fig = go.Figure()
for country in countries:
    fig.add_scatter(x=columns,
                    y=df_ratio.loc[country],
                    mode = 'lines',
                    name=country)

fig.show()

In [ ]:
showings_reader.client.close()